In [10]:
import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go

def turtle_profit(bid1, bid2, average, num_simulations=10000):
    """
    Simulate the profit per Flipper from buying at bid1 (first bid) or bid2 (second bid)
    given a known average second bid (from other traders). 
    The Turtle's reserve price is modeled as a triangular distribution.
    
    - If your first bid is above the reserve, you pay bid1 per unit.
    - If not, and your second bid is above the reserve, your profit is scaled by
      a multiplier reflecting how close your bid is to the overall average.
    - Otherwise, no trade occurs.
    """
    profits = []
    for _ in range(num_simulations):
        # Reserve price distribution for Sea Turtle Flippers:
        # Assume reserve price is triangular, with lower bound = 290, mode = 310, upper bound = 320.
        reserve_price = np.random.triangular(left=290, mode=310, right=320)
        if bid1 >= reserve_price:
            profit = 320 - bid1
        elif bid2 >= reserve_price:
            # For the second bid, if your offer is below the average of second bids,
            # then your profit will be scaled down.
            multiplier = (320 - average) / (320 - bid2) if bid2 < average else 1
            profit = (320 - bid2) * multiplier
        else:
            profit = 0
        profits.append(profit)
    return np.mean(profits)

def find_optimal_bids(average):
    """
    Iterates over candidate bid values for first and second bids.
    Assumes sensible bid ranges for Sea Turtles: from 290 to 320 SeaShells, in 0.5 increments.
    It only considers second bids that are at least the first bid.
    Returns the optimal first and second bids and the corresponding maximum expected profit.
    """
    max_expected_profit = -np.inf
    optimal_bid1 = None
    optimal_bid2 = None
    bid1_range = np.arange(290, 320.1, 0.25)  # candidate first bids
    bid2_range = np.arange(290, 320.1, 0.25)  # candidate second bids
    expected_profits = np.zeros((len(bid1_range), len(bid2_range)))
    
    for i, bid1 in enumerate(tqdm(bid1_range, desc="Searching bid1 range")):
        for j, bid2 in enumerate(bid2_range):
            if bid2 >= bid1:  # Typically, second bid must be at least as high as the first
                expected_profit = turtle_profit(bid1, bid2, average, num_simulations=15000)
                expected_profits[i, j] = expected_profit
                if expected_profit > max_expected_profit:
                    max_expected_profit = expected_profit
                    optimal_bid1 = bid1
                    optimal_bid2 = bid2
    return optimal_bid1, optimal_bid2, max_expected_profit, expected_profits

In [11]:
average_second_bid = 315

optimal_bid1, optimal_bid2, max_expected_profit, expected_profits = find_optimal_bids(average_second_bid)

print("Optimal First Bid:", optimal_bid1)
print("Optimal Second Bid:", optimal_bid2)
print("Maximum Expected Profit per Flipper:", max_expected_profit)

# Optionally, visualize the result in 3D:
fig = go.Figure(data=[go.Scatter3d(
    x=[result[0] for result in [(average_second_bid, optimal_bid1, optimal_bid2, max_expected_profit)]],  # constant in this case
    y=[optimal_bid1], 
    z=[optimal_bid2],
    mode='markers',
    marker=dict(
        size=10,
        color=[max_expected_profit],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=[f"Average: {average_second_bid}<br>Optimal Bid 1: {optimal_bid1}<br>Optimal Bid 2: {optimal_bid2}<br>Max Profit: {max_expected_profit:.2f}"],
    hoverinfo='text'
)])

fig.update_layout(
    title="Optimal Bids for Sea Turtles",
    scene=dict(
        xaxis_title="Average Second Bid",
        yaxis_title="Optimal First Bid",
        zaxis_title="Optimal Second Bid"
    )
)

fig.show()

Searching bid1 range: 100%|██████████| 121/121 [02:29<00:00,  1.24s/it]


Optimal First Bid: 307.25
Optimal Second Bid: 315.0
Maximum Expected Profit per Flipper: 8.521


In [12]:
def turtle_profit(bid1, bid2, average, num_simulations=10000, dist="triangular", dist_params=None, bid_noise=0):
    """
    Simulates the profit per Flipper under a given bidding strategy.

    Parameters:
      bid1: First bid (for immediate acceptance if above a turtle's reserve).
      bid2: Second bid (for acceptance if it is above the average of other second bids).
      average: The average second bid from other traders.
      num_simulations: Number of simulations to run.
      dist: The type of distribution to use for modeling reserve prices.
            Options: "triangular", "lognormal", "beta".
      dist_params: Dictionary of parameters for the chosen distribution.
         - For "triangular": keys are "left", "mode", "right". (e.g., { "left": 290, "mode": 310, "right": 320 })
         - For "lognormal": keys are "mu" and "sigma" (applied on log scale; default target around 310).
         - For "beta": keys are "low", "high", "a", and "b", where the generated beta value is scaled to [low, high].
      bid_noise: Standard deviation of a small Gaussian noise added to both bids to simulate variability.

    Returns:
      The mean profit per Flipper.
    """
    profits = []
    
    # Set defaults if no distribution parameters given:
    if dist_params is None:
        if dist == "triangular":
            dist_params = {"left": 290, "mode": 310, "right": 320}
        elif dist == "lognormal":
            # For lognormal, use mu so that the mean is near 310
            dist_params = {"mu": np.log(310), "sigma": 0.05}
        elif dist == "beta":
            dist_params = {"low": 290, "high": 320, "a": 2, "b": 5}
    
    for _ in range(num_simulations):
        # Draw a reserve price from the chosen distribution
        if dist == "triangular":
            reserve_price = np.random.triangular(dist_params["left"], dist_params["mode"], dist_params["right"])
        elif dist == "lognormal":
            reserve_price = np.random.lognormal(dist_params["mu"], dist_params["sigma"])
        elif dist == "beta":
            # Beta returns a value in [0, 1]; scale it to [low, high]
            beta_value = np.random.beta(dist_params["a"], dist_params["b"])
            reserve_price = dist_params["low"] + beta_value * (dist_params["high"] - dist_params["low"])
        else:
            raise ValueError("Unknown distribution type")
        
        # Optionally add noise to simulate bid variability
        bid1_effective = bid1 + np.random.normal(0, bid_noise)
        bid2_effective = bid2 + np.random.normal(0, bid_noise)
        
        # Calculate profit per flipper. In this round, you always eventually sell at 320 SeaShells.
        if bid1_effective >= reserve_price:
            profit = 320 - bid1_effective
        elif bid2_effective >= reserve_price:
            # For the second bid, if your bid is below the average, the profit is scaled down.
            multiplier = (320 - average) / (320 - bid2_effective) if bid2_effective < average else 1
            profit = (320 - bid2_effective) * multiplier
        else:
            profit = 0
        profits.append(profit)
    
    return np.mean(profits)


def find_optimal_bids(average, dist="triangular", dist_params=None, bid_noise=0, num_simulations=10000):
    max_expected_profit = -np.inf
    optimal_bid1 = None
    optimal_bid2 = None
    # Use a finer grid if needed for more rigor
    bid1_range = np.arange(290, 320.1, 0.5)
    bid2_range = np.arange(290, 320.1, 0.5)
    expected_profits = np.zeros((len(bid1_range), len(bid2_range)))
    
    for i, bid1 in enumerate(tqdm(bid1_range, desc="First bid range")):
        for j, bid2 in enumerate(bid2_range):
            if bid2 >= bid1:  # enforce that second bid is at least the first bid
                expected_profit = turtle_profit(bid1, bid2, average, num_simulations,
                                                dist=dist, dist_params=dist_params, bid_noise=bid_noise)
                expected_profits[i, j] = expected_profit
                if expected_profit > max_expected_profit:
                    max_expected_profit = expected_profit
                    optimal_bid1 = bid1
                    optimal_bid2 = bid2
    return optimal_bid1, optimal_bid2, max_expected_profit, expected_profits



In [14]:
# Example usage:
# Let's say the average second bid is estimated at 315 SeaShells
average_second_bid = 315
# You can start with the default triangular model or experiment with other distributions
optimal_bid1, optimal_bid2, max_expected_profit, expected_profits = find_optimal_bids(
    average_second_bid,
    dist="triangular",  # or "lognormal", "beta"
    dist_params={"left": 290, "mode": 310, "right": 320},
    bid_noise=0.5,  # a small noise to simulate bid variability
    num_simulations=15000  # more simulations for greater rigor
)

print("Optimal First Bid:", optimal_bid1)
print("Optimal Second Bid:", optimal_bid2)
print("Maximum Expected Profit per Flipper:", max_expected_profit)

# Optionally, visualize the results if you want:
mean_range = np.arange(290, 320.1, 0.5)
# (You could build a 3D plot similar to the goldfish example above to see how optimal bids vary with the average)


First bid range: 100%|██████████| 61/61 [01:27<00:00,  1.43s/it]

Optimal First Bid: 307.5
Optimal Second Bid: 315.0
Maximum Expected Profit per Flipper: 8.376429080755353


In [16]:
def turtle_profit(bid1, bid2, average, num_simulations=10000, 
                  dist="triangular", dist_params=None, bid_noise=0):
    """
    Simulate profit per Flipper.
    
    For each simulation:
      - Draw a Turtle's reserve price from a chosen distribution.
      - If your first bid (bid1) is above the reserve, you secure a trade and gain (320 - bid1).
      - Otherwise, if your second bid (bid2) is above the reserve, then you gain (320 - bid2) scaled by a multiplier,
        where the multiplier = (320 - average)/(320 - bid2) if bid2 < average, else 1.
      - Otherwise, no trade.
    """
    profits = []
    
    # Set default distribution parameters if none provided.
    if dist_params is None:
        if dist == "triangular":
            dist_params = {"left": 290, "mode": 310, "right": 320}
        elif dist == "lognormal":
            # Setting mu so that mean is near 310.
            dist_params = {"mu": np.log(310), "sigma": 0.05}
        elif dist == "beta":
            # Scale a beta distribution to [290,320]
            dist_params = {"low": 290, "high": 320, "a": 2, "b": 5}
    
    for _ in range(num_simulations):
        # Draw Turtle's reserve price from the chosen distribution.
        if dist == "triangular":
            reserve_price = np.random.triangular(dist_params["left"], dist_params["mode"], dist_params["right"])
        elif dist == "lognormal":
            reserve_price = np.random.lognormal(dist_params["mu"], dist_params["sigma"])
        elif dist == "beta":
            beta_val = np.random.beta(dist_params["a"], dist_params["b"])
            reserve_price = dist_params["low"] + beta_val * (dist_params["high"] - dist_params["low"])
        else:
            raise ValueError("Unknown distribution type")
        
        # Add randomness to bids to simulate competitor variability.
        bid1_effective = bid1 + np.random.normal(0, bid_noise)
        bid2_effective = bid2 + np.random.normal(0, bid_noise)
        
        # Determine profit:
        if bid1_effective >= reserve_price:
            profit = 320 - bid1_effective
        elif bid2_effective >= reserve_price:
            multiplier = (320 - average) / (320 - bid2_effective) if bid2_effective < average else 1
            profit = (320 - bid2_effective) * multiplier
        else:
            profit = 0
        profits.append(profit)
    
    return np.mean(profits)

def find_optimal_bids(average, dist="triangular", dist_params=None, bid_noise=0, num_simulations=10000):
    max_expected_profit = -np.inf
    optimal_bid1 = None
    optimal_bid2 = None
    bid1_range = np.arange(290, 320.1, 0.5)
    bid2_range = np.arange(290, 320.1, 0.5)
    expected_profits = np.zeros((len(bid1_range), len(bid2_range)))
    
    for i, bid1 in enumerate(tqdm(bid1_range, desc="Searching first bid range")):
        for j, bid2 in enumerate(bid2_range):
            if bid2 >= bid1:
                expected_profit = turtle_profit(bid1, bid2, average, num_simulations,
                                                dist=dist, dist_params=dist_params, bid_noise=bid_noise)
                expected_profits[i, j] = expected_profit
                if expected_profit > max_expected_profit:
                    max_expected_profit = expected_profit
                    optimal_bid1 = bid1
                    optimal_bid2 = bid2
    return optimal_bid1, optimal_bid2, max_expected_profit, expected_profits


average_second_bid = 315
optimal_bid1, optimal_bid2, max_expected_profit, expected_profits = find_optimal_bids(
    average_second_bid,
    dist="triangular",  
    dist_params={"left": 290, "mode": 310, "right": 320},
    bid_noise=0.5, 
    num_simulations=15000  # More simulations for increased accuracy
)

print("Optimal First Bid:", optimal_bid1)
print("Optimal Second Bid:", optimal_bid2)
print("Maximum Expected Profit per Flipper:", max_expected_profit)



Searching first bid range: 100%|██████████| 61/61 [01:16<00:00,  1.25s/it]


Optimal First Bid: 306.5
Optimal Second Bid: 315.0
Maximum Expected Profit per Flipper: 8.348534868083147


In [17]:

x_data = [average_second_bid] * (len(np.ravel(expected_profits)))
y_data = np.repeat(np.arange(290, 320.1, 0.5), len(np.arange(290, 320.1, 0.5)))
z_data = np.tile(np.arange(290, 320.1, 0.5), len(np.arange(290, 320.1, 0.5)))

fig = go.Figure(data=[go.Scatter3d(
    x=x_data,
    y=y_data,
    z=z_data,
    mode='markers',
    marker=dict(
        size=4,
        color=np.ravel(expected_profits),
        colorscale='Viridis',
        opacity=0.8
    ),
    text=["Profit: {:.2f}".format(profit) for profit in np.ravel(expected_profits)],
    hoverinfo='text'
)])

fig.update_layout(
    title="Optimal Bid Simulation for Sea Turtles",
    scene=dict(
        xaxis_title="Average Second Bid",
        yaxis_title="Optimal First Bid",
        zaxis_title="Optimal Second Bid"
    )
)

fig.show()


In [18]:
def turtle_profit(bid1, bid2, average, num_simulations=10000, bid_noise=0):
    """
    Simulates profit per Flipper over a number of iterations.
    
    Parameters:
      bid1: First bid (the bid that wins if above the Turtle's reserve).
      bid2: Second bid (if first bid fails). For second bid:
            - If bid2 >= reserve AND bid2 >= average, profit = 320 - bid2.
            - Else if bid2 >= reserve but bid2 < average, profit is scaled by:
                p = ((320 - average)/(320 - bid2))^3.
      average: The average second bid from other traders.
      num_simulations: Number of simulation iterations.
      bid_noise: Standard deviation of Gaussian noise added to bids.
      
    Returns:
      The average profit per Flipper from the simulation.
    """
    profits = []
    for _ in range(num_simulations):
        # Randomly choose which reserve distribution to use (50% chance each)
        if np.random.rand() < 0.5:
            reserve_price = np.random.uniform(160, 200)
        else:
            reserve_price = np.random.uniform(250, 320)
            
        # Add noise to your bids to simulate variability
        bid1_effective = bid1 + np.random.normal(0, bid_noise)
        bid2_effective = bid2 + np.random.normal(0, bid_noise)
        
        # Determine profit:
        if bid1_effective >= reserve_price:
            profit = 320 - bid1_effective
        elif bid2_effective >= reserve_price:
            # Check if bid2 is below the average second bid.
            if bid2_effective < average:
                p = ((320 - average) / (320 - bid2_effective)) ** 3
            else:
                p = 1
            profit = (320 - bid2_effective) * p
        else:
            profit = 0
        profits.append(profit)
    return np.mean(profits)


def find_optimal_bids(average, bid_noise=0, num_simulations=10000):
    max_expected_profit = -np.inf
    optimal_bid1 = None
    optimal_bid2 = None
    bid1_range = np.arange(160, 320.1, 1)  # you may restrict this further if desired
    bid2_range = np.arange(160, 320.1, 1)
    expected_profits = np.zeros((len(bid1_range), len(bid2_range)))
    
    for i, bid1 in enumerate(tqdm(bid1_range, desc="Searching bid1 range")):
        for j, bid2 in enumerate(bid2_range):
            if bid2 >= bid1:  # Ensure second bid is not lower than first
                exp_profit = turtle_profit(bid1, bid2, average, num_simulations=num_simulations, bid_noise=bid_noise)
                expected_profits[i, j] = exp_profit
                if exp_profit > max_expected_profit:
                    max_expected_profit = exp_profit
                    optimal_bid1 = bid1
                    optimal_bid2 = bid2
    return optimal_bid1, optimal_bid2, max_expected_profit, expected_profits


In [19]:
# %% [markdown]
# ## Run the Optimization
# 
# Let's say the average second bid from other traders is estimated to be 315 SeaShells.
# We run the simulation over a range of candidate bid values.

# %% [code]
average_second_bid = 315
optimal_bid1, optimal_bid2, max_expected_profit, expected_profits = find_optimal_bids(
    average_second_bid,
    bid_noise=1,         # Adjust noise as needed for your market environment
    num_simulations=15000
)

print("Optimal First Bid:", optimal_bid1)
print("Optimal Second Bid:", optimal_bid2)
print("Maximum Expected Profit per Flipper:", max_expected_profit)



Searching bid1 range: 100%|██████████| 161/161 [11:00<00:00,  4.10s/it]


Optimal First Bid: 200.0
Optimal Second Bid: 311.0
Maximum Expected Profit per Flipper: 61.43122142665145


In [20]:
# %% [markdown]
# ## 3D Visualization (Optional)
# 
# We can create a 3D scatter plot to visualize how the expected profit changes with your bid choices.

# %% [code]
x_data = [average_second_bid] * (len(np.ravel(expected_profits)))
y_data = np.repeat(np.arange(160, 320+1, 1), len(np.arange(160, 320+1, 1)))
z_data = np.tile(np.arange(160, 320+1, 1), len(np.arange(160, 320+1, 1)))

fig = go.Figure(data=[go.Scatter3d(
    x=x_data,
    y=y_data,
    z=z_data,
    mode='markers',
    marker=dict(
        size=4,
        color=np.ravel(expected_profits),
        colorscale='Viridis',
        opacity=0.8
    ),
    text=["Profit: {:.2f}".format(profit) for profit in np.ravel(expected_profits)],
    hoverinfo='text'
)])

fig.update_layout(
    title="Optimal Bid Simulation for Sea Turtles",
    scene=dict(
        xaxis_title="Average Second Bid",
        yaxis_title="Optimal First Bid",
        zaxis_title="Optimal Second Bid"
    )
)

fig.show()
